# 100 Coding Samples: Food & Cuisines EDA (Kaggle Edition)

This notebook demonstrates 100 samples of Exploratory Data Analysis (EDA) using datasets downloaded directly from Kaggle.

**Prerequisites:**
1. You must have a Kaggle account.
2. Create a New API Token in your Kaggle account settings to download `kaggle.json`.
3. Run the first cell below to upload that file.

## Part 1: Kaggle Setup & Data Download (Samples 1-5)

In [ ]:
# Sample 1: Upload Kaggle API Key
# Explanation: Uses Google Colab's file upload feature to let you upload your kaggle.json API key.
from google.colab import files
import os

# Check if kaggle.json is already uploaded
if not os.path.exists('kaggle.json'):
    print('Please upload your kaggle.json file:')
    files.upload()

In [ ]:
# Sample 2: Configure Kaggle Permissions
# Explanation: Moves the API key to the hidden .kaggle folder and secures it (chmod 600) so the API works.
import os
# Move key to correct location and set permissions
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
print('Kaggle API Key configured.')

In [ ]:
# Sample 3: Download Datasets
# Explanation: Uses the Kaggle CLI to download the Indian Food, Ramen Ratings, and Starbucks Menu datasets.
!kaggle datasets download -d nehaprabhavalkar/indian-food-101
!kaggle datasets download -d residentmario/ramen-ratings
!kaggle datasets download -d starbucks/starbucks-menu

In [ ]:
# Sample 4: Unzip Files
# Explanation: Extracts the CSV files from the downloaded ZIP archives.
import zipfile

# Unzipping the datasets
datasets = ['indian-food-101.zip', 'ramen-ratings.zip', 'starbucks-menu.zip']
for data in datasets:
    with zipfile.ZipFile(data, 'r') as zip_ref:
        zip_ref.extractall()
print('Datasets unzipped successfully.')

In [ ]:
# Sample 5: Load Data into Pandas
# Explanation: Reads the extracted CSV files into Pandas DataFrames for analysis.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Load the CSVs
df_indian = pd.read_csv('indian_food.csv')
df_ramen = pd.read_csv('ramen-ratings.csv')
df_starbucks = pd.read_csv('starbucks_drinkMenu_expanded.csv')

print('Data Loaded Successfully!')

## Part 2: Data Cleaning & Pre-processing (Samples 6-20)

In [ ]:
# Sample 6: Standardize Column Names
# Explanation: Critical Step: Cleans column names (lowercase, no spaces) so we don't get errors accessing them.
# Standardizing column names to avoid key errors later
df_starbucks.columns = [c.strip().replace(' ', '_').lower() for c in df_starbucks.columns]
df_ramen.columns = [c.strip().replace(' ', '_').lower() for c in df_ramen.columns]
df_indian.columns = [c.strip().replace(' ', '_').lower() for c in df_indian.columns]

# Rename specific columns for ease of use
df_starbucks.rename(columns={' total_fat_(g)': 'fat', 'sugars_(g)': 'sugar', 'caffeine_(mg)': 'caffeine'}, inplace=True)
print(df_starbucks.columns)

In [ ]:
# Sample 7: Handle Indian Food Placeholders
# Explanation: The Indian dataset uses '-1' for missing data. We convert these to true NaNs.
df_indian = df_indian.replace(-1, np.nan).replace('-1', np.nan)
df_indian.isnull().sum()

In [ ]:
# Sample 8: Clean Ramen Ratings
# Explanation: Converts 'stars' to numeric, turning 'Unrated' text into NaN, and drops those rows.
df_ramen['stars'] = pd.to_numeric(df_ramen['stars'], errors='coerce')
df_ramen.dropna(subset=['stars'], inplace=True)

In [ ]:
# Sample 9: Lowercase Ingredients
# Explanation: Ensures all ingredients are lowercase for accurate text analysis.
df_indian['ingredients'] = df_indian['ingredients'].str.lower().str.strip()

In [ ]:
# Sample 10: Remove Duplicates
# Explanation: Drops duplicate rows to prevent skewed statistics.
df_starbucks.drop_duplicates(inplace=True)
print(f'Starbucks Rows: {df_starbucks.shape[0]}')

In [ ]:
# Sample 11: Impute Missing Times
# Explanation: Fills missing cooking/prep times with the median value.
df_indian['cook_time'].fillna(df_indian['cook_time'].median(), inplace=True)
df_indian['prep_time'].fillna(df_indian['prep_time'].median(), inplace=True)

In [ ]:
# Sample 12: Standardize Country Names
# Explanation: Fixes inconsistent naming (USA vs United States).
df_ramen['country'] = df_ramen['country'].replace('USA', 'United States')

In [ ]:
# Sample 13: Clean Caffeine Column
# Explanation: Starbucks caffeine data often has text or NaNs. We force it to numeric and assume 0 for missing.
df_starbucks['caffeine'].fillna(0, inplace=True)
df_starbucks['caffeine'] = pd.to_numeric(df_starbucks['caffeine'], errors='coerce').fillna(0)

In [ ]:
# Sample 14: Feature Engineering (Count)
# Explanation: Creates a new column counting the number of ingredients per dish.
df_indian['num_ingredients'] = df_indian['ingredients'].apply(lambda x: len(str(x).split(',')) if pd.notnull(x) else 0)

In [ ]:
# Sample 15: Check Ramen Styles
# Explanation: Inspecting the 'style' column (Cup, Bowl, Pack) which relates to the packaging/comfort request.
df_ramen['style'].unique()

In [ ]:
# Sample 16: Ensure Calories are Numeric
# Explanation: Sometimes calories are stored as strings. We convert them.
df_starbucks['calories'] = pd.to_numeric(df_starbucks['calories'], errors='coerce')
df_starbucks.dropna(subset=['calories'], inplace=True)

In [ ]:
# Sample 17: Identify Outliers
# Explanation: Finding dishes with suspicious cook times (over 500 minutes).
outliers = df_indian[df_indian['cook_time'] > 500]
print(f'Dropping {len(outliers)} outliers')

In [ ]:
# Sample 18: Remove Outliers
# Explanation: Filtering the dataset to remove the extreme outliers found above.
df_indian = df_indian[df_indian['cook_time'] <= 500]

In [ ]:
# Sample 19: Fill Text NaNs
# Explanation: Filling missing Ramen variety names with a placeholder.
df_ramen['variety'] = df_ramen['variety'].fillna('Unknown')

In [ ]:
# Sample 20: Final Data Check
# Explanation: Reviewing the cleaned datasets before starting visualization.
df_starbucks.info()

## Part 3: Charts & Visualizations (Samples 21-50)

In [ ]:
# Sample 21: Histogram (Ramen Stars)
# Explanation: Shows the frequency distribution of ratings. Most ramen is rated highly (3-5 stars).
sns.set_theme(style='whitegrid')
plt.figure(figsize=(8,5))
sns.histplot(df_ramen['stars'], bins=10, kde=True, color='purple')
plt.title('Distribution of Ramen Ratings')
plt.show()

In [ ]:
# Sample 22: Count Plot
# Explanation: Visualizing the balance of diet types in Indian cuisine.
plt.figure(figsize=(10,6))
sns.countplot(y='diet', data=df_indian, palette='Set2')
plt.title('Vegetarian vs Non-Vegetarian Indian Dishes')
plt.show()

In [ ]:
# Sample 23: Bar Chart (Packaging)
# Explanation: This addresses the 'cushion/comfort' proxy—how the food is packaged for consumption.
plt.figure(figsize=(8,6))
sns.countplot(x='style', data=df_ramen)
plt.title('Ramen Packaging Styles (Cup vs Bowl vs Pack)')
plt.show()

In [ ]:
# Sample 24: Horizontal Bar Plot
# Explanation: Which countries produce the most distinct types of ramen?
top_countries = df_ramen['country'].value_counts().head(10)
plt.figure(figsize=(10,5))
sns.barplot(x=top_countries.values, y=top_countries.index, palette='viridis')
plt.title('Top 10 Countries by Ramen Variety')
plt.show()

In [ ]:
# Sample 25: Pie Chart
# Explanation: Shows the breakdown of dishes into Main Course, Dessert, Snack, etc.
plt.figure(figsize=(8,8))
df_indian['course'].value_counts().plot.pie(autopct='%1.1f%%')
plt.title('Meal Course Distribution')
plt.ylabel('')
plt.show()

In [ ]:
# Sample 26: Boxplot (Calories)
# Explanation: Visualizing the spread of calories to spot median values and high-calorie outliers.
plt.figure(figsize=(10,6))
sns.boxplot(x='calories', data=df_starbucks, color='lightblue')
plt.title('Boxplot of Starbucks Calories')
plt.show()

In [ ]:
# Sample 27: Violin Plot
# Explanation: Shows the density of sugar content. Note the bulge where most drinks cluster.
plt.figure(figsize=(10,6))
sns.violinplot(x='sugar', data=df_starbucks, color='pink')
plt.title('Violin Plot of Sugar Content')
plt.show()

In [ ]:
# Sample 28: Pandas Bar Chart
# Explanation: Quick visualization of flavor profiles (Spicy, Sweet, Bitter, etc.).
df_indian['flavor_profile'].value_counts().plot(kind='bar', color='teal')
plt.title('Flavor Profiles')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Sample 29: KDE Plot
# Explanation: Kernel Density Estimate showing the probability of different preparation times.
sns.kdeplot(data=df_indian, x='prep_time', fill=True, clip=(0,100))
plt.title('Prep Time Density (Clipped < 100 min)')
plt.show()

In [ ]:
# Sample 30: Interactive Plotly Histogram
# Explanation: Hover over the bars to see exact counts of drinks at different caffeine levels.
fig = px.histogram(df_starbucks, x='caffeine', nbins=30, title='Interactive Caffeine Histogram')
fig.show()

In [ ]:
# Sample 31: Strip Plot
# Explanation: Plots every single data point, useful for seeing sample size per category.
sns.stripplot(x='course', y='cook_time', data=df_indian, jitter=True)
plt.title('Strip Plot: Cook Time by Course')
plt.show()

In [ ]:
# Sample 32: Aggregated Bar Chart
# Explanation: Grouping by country to find quality rather than quantity.
avg_rating = df_ramen.groupby('country')['stars'].mean().sort_values(ascending=False).head(10)
avg_rating.plot(kind='bar', color='gold')
plt.title('Countries with Highest Avg Ramen Ratings')
plt.ylim(3,5)
plt.show()

In [ ]:
# Sample 33: ECDF Plot
# Explanation: Cumulative distribution: 'What % of drinks are under X calories?'
sns.ecdfplot(data=df_starbucks, x='calories')
plt.title('ECDF of Calories')
plt.grid(True)
plt.show()

In [ ]:
# Sample 34: Stacked Histogram
# Explanation: Comparing cook times of Veg vs Non-Veg visually.
sns.histplot(data=df_indian, x='cook_time', hue='diet', multiple='stack', bins=15)
plt.xlim(0, 150)
plt.title('Stacked Histogram: Cook Time by Diet')
plt.show()

In [ ]:
# Sample 35: Funnel Chart
# Explanation: Visualizing the hierarchy of regions contributing to the dataset.
regions = df_indian['region'].value_counts()
fig = px.funnel(regions, title='Funnel: Dishes by Region')
fig.show()

In [ ]:
# Sample 36: Donut Chart
# Explanation: A cleaner alternative to the pie chart for showing part-to-whole relationships.
plt.figure(figsize=(7,7))
plt.pie(df_ramen['style'].value_counts(), labels=df_ramen['style'].value_counts().index, autopct='%1.1f%%')
plt.gca().add_artist(plt.Circle((0,0),0.70,fc='white'))
plt.title('Donut Chart: Ramen Styles')
plt.show()

In [ ]:
# Sample 37: Rug Plot
# Explanation: Marks ticks on the axis for every data point, showing density.
sns.rugplot(data=df_starbucks, x='fat', height=.1, color='red')
plt.title('Rug Plot: Fat Content')
plt.show()

In [ ]:
# Sample 38: Heatmap
# Explanation: A matrix visualization showing which Region+Course combos take the longest to prep.
pivot = df_indian.pivot_table(index='region', columns='course', values='prep_time', aggfunc='mean')
sns.heatmap(pivot, annot=True, cmap='coolwarm', fmt='.0f')
plt.title('Heatmap: Avg Prep Time')
plt.show()

In [ ]:
# Sample 39: Interactive Scatter Plot
# Explanation: Shows the correlation between sugar and calories, colored by drink category.
fig = px.scatter(df_starbucks, x='sugar', y='calories', color='beverage_category', title='Scatter: Sugar vs Calories')
fig.show()

In [ ]:
# Sample 40: Hexbin Joint Plot
# Explanation: Shows where the mass of data points sits for prep vs cook time.
sns.jointplot(data=df_indian, x='prep_time', y='cook_time', kind='hex', xlim=(0,60), ylim=(0,60))
plt.show()

In [ ]:
# Sample 41: Interactive Box Plot
# Explanation: Compare the rating distributions of Cups vs Packs vs Bowls.
fig = px.box(df_ramen, x='style', y='stars', color='style', title='Box Plot: Stars by Style')
fig.show()

In [ ]:
# Sample 42: Point Plot
# Explanation: Good for comparing changes or differences between categorical variables.
sns.pointplot(x='course', y='cook_time', hue='diet', data=df_indian)
plt.title('Point Plot: Cook Time Trends')
plt.show()

In [ ]:
# Sample 43: Regression Plot
# Explanation: Fits a trend line to the scatter plot.
sns.lmplot(x='calories', y='fat', data=df_starbucks, height=5, aspect=1.5)
plt.title('Linear Regression: Calories vs Fat')
plt.show()

In [ ]:
# Sample 44: Swarm Plot
# Explanation: Like a strip plot, but points are adjusted so they don't overlap.
sns.swarmplot(x='diet', y='prep_time', data=df_indian, size=3)
plt.ylim(0, 100)
plt.title('Swarm Plot: Prep Time')
plt.show()

In [ ]:
# Sample 45: Treemap
# Explanation: A hierarchical view of the data. Click segments to zoom in.
fig = px.treemap(df_indian, path=['region', 'state', 'course'], title='Treemap: Indian Cuisine')
fig.show()

In [ ]:
# Sample 46: Correlation Matrix
# Explanation: Shows how strongly nutritional variables are related (e.g., Fat vs Calories).
corr = df_starbucks[['calories', 'fat', 'sugar', 'caffeine']].corr()
sns.heatmap(corr, annot=True, cmap='Reds')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# Sample 47: Sunburst Chart
# Explanation: A radial hierarchy chart. Excellent for exploring nested categories.
fig = px.sunburst(df_indian, path=['diet', 'course', 'flavor_profile'], title='Sunburst: Diet > Course > Flavor')
fig.show()

In [ ]:
# Sample 48: Pair Plot
# Explanation: Creates a grid of scatter plots for every pair of selected variables.
sns.pairplot(df_starbucks[['calories', 'sugar', 'caffeine']])
plt.show()

In [ ]:
# Sample 49: Facet Grid
# Explanation: Small multiples: Separate histograms for each ramen style.
g = sns.FacetGrid(df_ramen, col='style', col_wrap=4)
g.map(sns.histplot, 'stars')
plt.show()

In [ ]:
# Sample 50: 3D Scatter Plot
# Explanation: Visualizing three variables at once in 3D space.
fig = px.scatter_3d(df_starbucks, x='sugar', y='calories', z='caffeine', color='beverage_category', title='3D Scatter')
fig.show()

## Part 4: Advanced Analysis (Samples 51-100)

In [ ]:
# Sample 51: Word Cloud
# Explanation: Visualizing the most common text words in the ingredients column.
from wordcloud import WordCloud
text = ' '.join(df_indian['ingredients'])
wc = WordCloud(width=800, height=400, background_color='white').generate(text)
plt.imshow(wc)
plt.axis('off')
plt.title('Word Cloud: Indian Ingredients')
plt.show()

In [ ]:
# Sample 52: Feature Engineering (Text)
# Explanation: Do ramen brands have short or long names?
df_ramen['brand_len'] = df_ramen['brand'].apply(lambda x: len(str(x)))
sns.histplot(df_ramen['brand_len'])
plt.title('Brand Name Lengths')
plt.show()

In [ ]:
# Sample 53: Text Frequency Count
# Explanation: Mathematically counting the most frequent ingredients.
top_ing = pd.Series(' '.join(df_indian['ingredients']).split()).value_counts().head(10)
print(top_ing)

In [ ]:
# Sample 54: Bar Chart of Text Data
# Explanation: Plotting the ingredient counts from Sample 53.
fig = px.bar(top_ing, title='Top 10 Ingredients')
fig.show()

In [ ]:
# Sample 55: String Search
# Explanation: Calculating the prevalence of 'Spicy' ramen.
spicy_pct = len(df_ramen[df_ramen['variety'].str.contains('Spicy', case=False)]) / len(df_ramen) * 100
print(f'Spicy Ramen Percentage: {spicy_pct:.2f}%')

In [ ]:
# Sample 56: Comparative KDE
# Explanation: Comparing rating distributions based on a text keyword.
sns.kdeplot(df_ramen[df_ramen['variety'].str.contains('Spicy', case=False)]['stars'], label='Spicy')
sns.kdeplot(df_ramen[~df_ramen['variety'].str.contains('Spicy', case=False)]['stars'], label='Non-Spicy')
plt.legend()
plt.title('Ratings: Spicy vs Non-Spicy')
plt.show()

In [ ]:
# Sample 57: Ratio Analysis
# Explanation: Which drink gives the most caffeine per calorie?
df_starbucks['caffeine_ratio'] = df_starbucks['caffeine'] / (df_starbucks['calories'] + 1)
df_starbucks.sort_values('caffeine_ratio', ascending=False).head(3)[['beverage', 'caffeine_ratio']]

In [ ]:
# Sample 58: Grouped Bar Chart
# Explanation: A complex Pandas plot comparing multiple categories.
df_indian.groupby(['region', 'diet'])['cook_time'].mean().unstack().plot(kind='bar')
plt.title('Avg Cook Time by Region & Diet')
plt.show()

In [ ]:
# Sample 59: Parallel Categories
# Explanation: Visualizing the flow of data across categorical dimensions.
fig = px.parallel_categories(df_indian, dimensions=['region', 'course', 'diet'])
fig.show()

In [ ]:
# Sample 60: Boxen Plot
# Explanation: Enhanced box plot for larger datasets.
sns.boxenplot(x='country', y='stars', data=df_ramen[df_ramen['country'].isin(['Japan', 'USA', 'South Korea'])])
plt.title('Boxen Plot')
plt.show()

In [ ]:
# Sample 61: Total Time Calc
# Explanation: Summing prep and cook time.
df_indian['total_time'] = df_indian['prep_time'] + df_indian['cook_time']

In [ ]:
# Sample 62: Log Scale Plot
# Explanation: Handling skewed data.
sns.histplot(df_indian['total_time'], log_scale=True)
plt.title('Log Scaled Total Time')

In [ ]:
# Sample 63: Crosstab
# Explanation: Frequency table of Region vs Course.
pd.crosstab(df_indian['region'], df_indian['course'])

In [ ]:
# Sample 64: Heatmap of Crosstab
# Explanation: Visualizing the table above.
sns.heatmap(pd.crosstab(df_indian['region'], df_indian['course']), cmap='Greens')

In [ ]:
# Sample 65: Top 5 Sugar
# Explanation: Finding max sugar drinks.
df_starbucks.nlargest(5, 'sugar')[['beverage', 'sugar']]

In [ ]:
# Sample 66: Top 5 Low Cal
# Explanation: Finding diet drinks.
df_starbucks.nsmallest(5, 'calories')[['beverage', 'calories']]

In [ ]:
# Sample 67: Word Count
# Explanation: Counting words in ramen variety names.
df_ramen['word_count'] = df_ramen['variety'].apply(lambda x: len(str(x).split()))

In [ ]:
# Sample 68: Scatter Text vs Rating
# Explanation: Does name length affect rating?
sns.scatterplot(x='word_count', y='stars', data=df_ramen)

In [ ]:
# Sample 69: Filter (Milk)
# Explanation: Count dishes with milk.
df_indian[df_indian['ingredients'].str.contains('milk')].shape[0]

In [ ]:
# Sample 70: Filter (Rice)
# Explanation: Count dishes with rice.
df_indian[df_indian['ingredients'].str.contains('rice')].shape[0]

In [ ]:
# Sample 71: Avg Cal by Category
# Explanation: Which category is heaviest?
df_starbucks.groupby('beverage_category')['calories'].mean().plot(kind='barh')

In [ ]:
# Sample 72: Clustermap
# Explanation: Hierarchical clustering of correlations.
sns.clustermap(corr, figsize=(5,5))

In [ ]:
# Sample 73: Cumulative Sum
# Explanation: Pareto-style analysis of countries.
df_ramen['country'].value_counts().cumsum().plot()

In [ ]:
# Sample 74: Count Unique
# Explanation: How many states are represented?
df_indian['state'].nunique()

In [ ]:
# Sample 75: 5-Star Sources
# Explanation: Where does the best ramen come from?
df_ramen[df_ramen['stars'] == 5]['country'].value_counts().head()

In [ ]:
# Sample 76: Scatter Matrix
# Explanation: Interactive pair plot.
fig = px.scatter_matrix(df_starbucks, dimensions=['calories', 'sugar', 'caffeine'])

In [ ]:
# Sample 77: Describe Stats
# Explanation: Full statistical summary.
df_indian['cook_time'].describe()

In [ ]:
# Sample 78: Top Brands Pie
# Explanation: Market share of brands.
df_ramen['brand'].value_counts().head(5).plot(kind='pie')

In [ ]:
# Sample 79: Residual Plot
# Explanation: Checking regression fit.
sns.residplot(x='sugar', y='calories', data=df_starbucks)

In [ ]:
# Sample 80: Aggregations
# Explanation: Multi-stat groupby.
df_indian.groupby('diet')['prep_time'].agg(['mean', 'max'])

In [ ]:
# Sample 81: Split Violin
# Explanation: Comparing distributions side-by-side.
sns.violinplot(x='course', y='prep_time', hue='diet', data=df_indian, split=True)

In [ ]:
# Sample 82: Binary Feature
# Explanation: Flagging whipped cream.
df_starbucks['has_whip'] = df_starbucks['beverage'].apply(lambda x: 1 if 'whip' in str(x).lower() else 0)

In [ ]:
# Sample 83: Whip Impact
# Explanation: Does whip add significantly to calories?
sns.barplot(x='has_whip', y='calories', data=df_starbucks)

In [ ]:
# Sample 84: Date Gen
# Explanation: Simulating a time series.
df_ramen['dummy_date'] = pd.date_range(start='1/1/2022', periods=len(df_ramen))

In [ ]:
# Sample 85: Multi-Sort
# Explanation: Quickest overall dishes.
df_indian.sort_values(['prep_time', 'cook_time']).head()

In [ ]:
# Sample 86: Column Means
# Explanation: Average of all numeric columns.
df_starbucks.select_dtypes('number').mean()

In [ ]:
# Sample 87: Fillna Count
# Explanation: Counting with missing values included.
df_indian['region'].fillna('Unknown').value_counts()

In [ ]:
# Sample 88: CDF
# Explanation: Cumulative density of calories.
sns.kdeplot(df_starbucks['calories'], cumulative=True)

In [ ]:
# Sample 89: Binning
# Explanation: Categorizing continuous ratings.
df_ramen['tier'] = pd.cut(df_ramen['stars'], bins=[0,3,4,5], labels=['Low','Mid','High'])

In [ ]:
# Sample 90: Plot Bins
# Explanation: Visualizing tiers.
sns.countplot(x='tier', data=df_ramen)

In [ ]:
# Sample 91: Pivot Table
# Explanation: Summarizing cook times.
pd.pivot_table(df_indian, index='region', columns='diet', values='cook_time')

In [ ]:
# Sample 92: Column Filter
# Explanation: Get columns with 'g' (grams).
df_starbucks.filter(like='g').head()

In [ ]:
# Sample 93: Query
# Explanation: Finding super quick meals.
df_indian.query('prep_time < 15 and cook_time < 15')

In [ ]:
# Sample 94: Count Unique
# Explanation: How many brands?
df_ramen['brand'].nunique()

In [ ]:
# Sample 95: Line Plot
# Explanation: Trend of sorted calories.
sns.lineplot(data=df_starbucks.sort_values('calories').reset_index(drop=True)['calories'])

In [ ]:
# Sample 96: Interactive Bar
# Explanation: Plotly stacked bar.
fig=px.bar(df_indian, x='state', y='prep_time', color='diet', title='Stacked Bar')

In [ ]:
# Sample 97: Text Startswith
# Explanation: Dishes starting with 'A'.
df_indian['name'].str.startswith('A').sum()

In [ ]:
# Sample 98: Agg Table
# Explanation: Stats per ramen style.
df_ramen.groupby('style')['stars'].agg(['mean','count'])

In [ ]:
# Sample 99: Corr Heatmap
# Explanation: Correlation of time variables.
sns.heatmap(df_indian.corr(numeric_only=True), annot=True)

In [ ]:
# Sample 100: Done
# Explanation: Finished.
print('100 Samples Completed!')